In [3]:
cd /Users/yyw/Desktop/python_3.9

/Users/yyw/Desktop/python_3.9


In [1]:
import yfinance as yf
import pandas as pd
from functools import reduce
from pandas_datareader import data
import pandas_datareader as web
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt        
%matplotlib inline

In [6]:
ticker = ['AMZN','GOOG','ZM','EPD','^GSPC','^IRX']
def getYData(ticker,period='1d',start='2010-1-1',end='2020-3-13'):
    get_data = web.get_data_yahoo(ticker,start,end)
    return get_data['Close']
raw_data=getYData(ticker)
raw_data

Symbols,AMZN,GOOG,ZM,EPD,^GSPC,^IRX
Date,,,,,,
2010-01-04,133.899994,312.204773,NaN,16.330000,1132.989990,0.055
2010-01-05,134.690002,310.829926,NaN,16.510000,1136.520020,0.060
2010-01-06,132.250000,302.994293,NaN,16.540001,1137.140015,0.045
2010-01-07,130.000000,295.940735,NaN,16.125000,1141.689941,0.045
2010-01-08,133.520004,299.885956,NaN,16.209999,1144.979980,0.040
...,...,...,...,...,...,...
2020-03-09,1800.609985,1215.560059,113.750000,18.250000,2746.560059,0.330
2020-03-10,1891.819946,1280.390015,108.489998,18.600000,2882.229980,0.390
2020-03-11,1820.859985,1215.410034,110.300003,17.209999,2741.379883,0.368


In [33]:
P = raw_data.pct_change(1)

In [50]:
cd /Users/yyw/Desktop/python_3.9

/Users/yyw/Desktop/python_3.9


In [51]:
pwd

'/Users/yyw/Desktop/python_3.9'

In [47]:
r = R['r']
r_rf = P['^IRX']/252
r_mkt = R['^GSPC']
#del R['^IRX']


KeyError: 'r'

In [46]:
def PrepareData(r,r_mkt,r_rf,tBreak = None):
    raw = [r,r_mkt,r_rf]
    R = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],how='outer'), raw)
    R.columns = ['r','r_mkt','r_rf']
    R = R.dropna(axis=0, how='any')
    #if tBreak != None:
        #R1 = R.loc[R.index =< tBreak]
        #R2 = R.loc[R.index > tBreak]
        #return([R1,R2])
    #else:
    return(R)
PrepareData('AMZN','^GSPC','^IRX')

UnboundLocalError: local variable 'R' referenced before assignment

In [7]:

R

Symbols,AMZN,GOOG,ZM,EPD,^GSPC,^IRX
Date,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.005900,-0.004404,NaN,0.011023,0.003116,0.090909
2010-01-06,-0.018116,-0.025209,NaN,0.001817,0.000546,-0.250000
2010-01-07,-0.017013,-0.023280,NaN,-0.025091,0.004001,0.000000
2010-01-08,0.027077,0.013331,NaN,0.005271,0.002882,-0.111111
...,...,...,...,...,...,...
2020-03-09,-0.052854,-0.063809,-0.004986,-0.185995,-0.075970,-0.204819
2020-03-10,0.050655,0.053333,-0.046242,0.019178,0.049396,0.181818
2020-03-11,-0.037509,-0.050750,0.016684,-0.074731,-0.048868,-0.056410


KeyError: 'r'

In [ ]:
R = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],how='outer'), raw)

In [ ]:
def getModel(r,r_mkt,r_rf,tBreak = None):
    R = PrepareData(r,r_mkt,r_rf)
    r = R['r']
    r_mkt = R['r_mkt']
    r_rf = R['r_rf']
    y = r - r_rf
    x = r_mkt - r_rf
    fit = sm.OLS(y,x).fit()
    return(fit)

In [ ]:
P.columns = tickerSymbols

In [ ]:
del P['^IRX']

In [ ]:
P.plot(kind='line')

In [ ]:
R = P.pct_change(1)

In [ ]:
R.plot(kind='line')

In [ ]:
import statsmodels.api as sm


In [ ]:
tBreak = '2020-02-14'

In [ ]:
def getBeta(r,r_mkt,r_rf):  
    fit = getModel(r,r_mkt,r_rf)
    out = pd.read_html(fit.summary().tables[1].as_html(),header=0,index_col=0)
    beta = out[0]['coef'][0]
    return(beta)

In [ ]:
#R.loc[R.index < '2020-02-16 08:00:00']

In [ ]:
def FitN4cast(r,r_mkt,r_rf,tBreak):
    R1,R2 = PrepareData(r,r_mkt,r_rf,tBreak)
    b = getBeta(R1['r'],r_mkt,r_rf)
    rhat = b*R2['r_mkt']
    r = R2['r']
    return([rhat,r])

In [ ]:
rhat,r = FitN4cast(r,r_mkt,r_rf,tBreak)

In [ ]:
d = {'rhat': rhat, 'r': r}
df = pd.DataFrame(data=d)
df.plot(kind='line')

In [ ]:
p_t = P.loc[P.index == '2020-02-14'].AMZN[0]

In [ ]:
phat = p_t * (1+rhat)